In [106]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

TESTING FOR GITHUB

In [107]:
df = pd.read_csv(r'C:\Users\JEFF\Downloads\playground-series-s4e12/train.csv')
df_test = pd.read_csv(r'C:\Users\JEFF\Downloads\playground-series-s4e12/test.csv')
df_sub = pd.read_csv(r'C:\Users\JEFF\Downloads\playground-series-s4e12/sample_submission.csv')

# Model Training
- Train test split
- Create sklearn pipeline
- Create pipelines for numerical and categorical values
- Use LGBMRegressor


In [108]:
X = df.drop(columns=['Premium Amount'])
y = df['Premium Amount']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=43)

In [109]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
import category_encoders as ce
from sklearn.metrics import mean_squared_log_error


In [110]:
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error, make_scorer

In [185]:
def rmsle(y_true, y_pred):
    # Clip both inputs to ensure non-negative values
    y_true_clipped = np.maximum(y_true, 0)
    y_pred_clipped = np.maximum(y_pred, 0)
    return np.sqrt(mean_squared_log_error(y_true_clipped, y_pred_clipped))

In [241]:
params = {
    'boosting_type': 'dart',
    'num_leaves': 384,
    'learning_rate': 0.024680120465142227,
    'feature_fraction': 0.9883068358315126,
    'bagging_fraction': 0.7201712704805496,
    'bagging_freq': 7,
    'min_data_in_leaf': 50,
    'max_depth': 15,
    'lambda_l1': 0.0011290211269753322,
    'lambda_l2': 3.056310541294088,
    'seed': 42
}
model = LGBMRegressor(**params)

In [113]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [114]:
df.head(5)

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [115]:
import seaborn as sns

    # Date features
    X['datetime'] = pd.to_datetime(X['Policy Start Date'])
    X['policy_date'] = X['datetime'].dt.date
    X['policy_start_year'] = X['datetime'].dt.year
    X['policy_start_month'] = X['datetime'].dt.month
    X['policy_start_day'] = X['datetime'].dt.day
    X['policy_start_dayofweek'] = X['datetime'].dt.dayofweek
    # Cyclical encoding for date features
    min_year = X['policy_start_year'].min()
    max_year = X['policy_start_year'].max()
    X['year_sin'] = np.sin(2*np.pi*(X['policy_start_year']-min_year))/(max_year-min_year)
    X['year_cos'] = np.cos(2*np.pi*(X['policy_start_year']-min_year))/(max_year-min_year)
    X['month_sin'] = np.sin(2*np.pi*(X['policy_start_month'])/12)
    X['month_cos'] = np.cos(2*np.pi*(X['policy_start_month'])/12)
    X['day_sin'] = np.sin(2*np.pi*(X['policy_start_day'])/31)
    X['day_cos'] = np.cos(2*np.pi*(X['policy_start_day'])/31)

    # Missing value flags
    for col in X.columns:
        X[f'{col}_missing'] = X[col].isna().astype(int)



    columns_with_nulls = df.columns[df.isnull().any()]
    X = df.copy()
    # Text features
    X['Feedback_Length'] = X['Customer Feedback'].astype(str).apply(len)
    X['Feedback_WordCount'] = X['Customer Feedback'].astype(str).apply(lambda x: len(x.split()))
    
    # Interaction features
    X['Age_Health'] = X['Age'] * X['Health Score']



    X.drop(columns=['id', 'Policy Start Date', 'datetime','policy_date'], inplace=True)
    columns_to_exclude=['id', 'Policy Start Date', 'datetime','Premium Amount','policy_date']

In [209]:
def get_column_types(df):
    cat_cols = []
    num_cols = []
    columns_to_exclude=['id', 'Policy Start Date', 'datetime','Premium Amount']
    for col in df.columns:
        if col not in columns_to_exclude:  # Only process columns we'll keep
            if df[col].dtypes == 'object':
                cat_cols.append(col)
            else:
                num_cols.append(col)
    return num_cols, cat_cols

# Then use them in your pipeline
num_cols, cat_cols = get_column_types(df)

In [146]:
df.columns

Index(['id', 'Age', 'Gender', 'Annual Income', 'Marital Status',
       'Number of Dependents', 'Education Level', 'Occupation', 'Health Score',
       'Location', 'Policy Type', 'Previous Claims', 'Vehicle Age',
       'Credit Score', 'Insurance Duration', 'Policy Start Date',
       'Customer Feedback', 'Smoking Status', 'Exercise Frequency',
       'Property Type', 'Premium Amount'],
      dtype='object')

In [138]:
score =[]

In [237]:
def feature_engineering(df):

    X = df.copy()

    # Interaction features
    X['Age_Health'] = X['Age'] * X['Health Score']
    
    # Date features
    X['datetime'] = pd.to_datetime(X['Policy Start Date'])
    X['policy_start_year'] = X['datetime'].dt.year
    X['policy_start_month'] = X['datetime'].dt.month
    X['policy_start_day'] = X['datetime'].dt.day
    X['policy_start_dayofweek'] = X['datetime'].dt.dayofweek
    X['policy_start_isweekend'] = X['datetime'].dt.weekday.isin([5, 6]).astype(int)

    # Cyclical encoding for date features
    min_year = X['policy_start_year'].min()
    max_year = X['policy_start_year'].max()
    X['year_sin'] = np.sin(2*np.pi*(X['policy_start_year']-min_year))/(max_year-min_year)
    X['year_cos'] = np.cos(2*np.pi*(X['policy_start_year']-min_year))/(max_year-min_year)
    X['month_sin'] = np.sin(2*np.pi*(X['policy_start_month'])/12)
    X['month_cos'] = np.cos(2*np.pi*(X['policy_start_month'])/12)
    X['day_sin'] = np.sin(2*np.pi*(X['policy_start_day'])/31)
    X['day_cos'] = np.cos(2*np.pi*(X['policy_start_day'])/31)


    # Text features
    X['Feedback_Length'] = X['Customer Feedback'].astype(str).apply(len)
    X['Feedback_WordCount'] = X['Customer Feedback'].astype(str).apply(lambda x: len(x.split()))
    
    # Missing value flags
    for col in X.columns:
        X[f'{col}_missing'] = X[col].isna().astype(int)

    # Drop unnecessary columns
    X.drop(columns=['id', 'Policy Start Date', 'datetime'], inplace=True)
    return X

In [242]:
num_pipeline = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

cat_pipeline = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='constant',fill_value='unknown')),
    ('TE',OneHotEncoder())
])

col_transformer = ColumnTransformer(transformers = [
    ('num_pipeline',num_pipeline, num_cols),
    ('cat_pipeline',cat_pipeline, cat_cols)
],
    remainder = 'passthrough',
    n_jobs = -1
)

pipeline = Pipeline(steps = [
    ('feature_engineering', FunctionTransformer(feature_engineering)),
    ('column_transformer', col_transformer),
    ('regressor',model)
])

In [243]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
rmsle_score = rmsle(y_test, y_pred)
score.append(rmsle_score)
print(f"RMSLE Score: {rmsle_score}")
print(score)

C:\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.9883068358315126, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9883068358315126
[LightGBM] [Warning] lambda_l1 is set=0.0011290211269753322, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0011290211269753322
[LightGBM] [Warning] lambda_l2 is set=3.056310541294088, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.056310541294088
[LightGBM] [Warning] bagging_fraction is set=0.7201712704805496, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7201712704805496
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.9883068358315126, colsample_b

C:\Python311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.9883068358315126, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9883068358315126
[LightGBM] [Warning] lambda_l1 is set=0.0011290211269753322, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0011290211269753322
[LightGBM] [Warning] lambda_l2 is set=3.056310541294088, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.056310541294088
[LightGBM] [Warning] bagging_fraction is set=0.7201712704805496, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7201712704805496
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
RMSLE Score: 1.0671030028158341
[1.1376161128067825, 1.1376161128067825, 1.134240726426666, 1.1344173332447585, 1.1342969484140044, 1.1342841205579517, 1.134263466211226, 1.1342622410379963, 1.13355

In [123]:
df_sub['Premium Amount'] = pipeline.predict(df_test)

In [124]:
df_sub.to_csv(r"C:\Users\JEFF\Downloads\playground-series-s4e12\submission.csv",index=False)